## Obtencion del dataset

### Paso 1: Instalar Dependencias para el proyecto

In [2]:
%pip install roboflow tensorflow

     ---------------------------------------- 85.4/85.4 KB 1.6 MB/s eta 0:00:00
     -------------------------------------- 375.7/375.7 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 128.7/128.7 KB 7.4 MB/s eta 0:00:00
     -------------------------------------- 162.3/162.3 KB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 66.8/66.8 KB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 38.8/38.8 MB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 KB ? eta 0:00:00
     ---------------------------------------- 8.6/8.6 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 54.5/54.5 KB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 KB ? eta 0:00:00
     -------------------------------------- 159.6/159.6 

You should consider upgrading via the 'C:\Users\mari_\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


### Paso 2: Descargar el dataset de Roboflow

In [3]:

from roboflow import Roboflow
rf = Roboflow(api_key="nK6spHh5hG9qjWGCbZ43")
project = rf.workspace("miguellopez").project("image-classification-jtsve")
version = project.version(1)
dataset = version.download("folder")
                
                

loading Roboflow workspace...
loading Roboflow project...


### Paso 3: Cargar dataset con ImageDataGenerator

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224,224)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% para entrenamiento, 20% para validación
)

train_generator = datagen.flow_from_directory(
    dataset.location + "/train",
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Conjunto de entrenamiento
)

validation_generator = datagen.flow_from_directory(
    dataset.location + "/train",
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Conjunto de validación
)


Found 1646 images belonging to 7 classes.
Found 409 images belonging to 7 classes.


### Paso 4: Definir el modelo CNN basico

In [31]:
# Paso 4: Definir modelo CNN básico
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


### Paso 5: Entrenar el modelo

In [32]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


52/52 ━━━━━━━━━━━━━━━━━━━━ 36s 664ms/step - accuracy: 0.2789 - loss: 3.9814 - val_accuracy: 0.1760 - val_loss: 2.4297
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 33s 643ms/step - accuracy: 0.5757 - loss: 1.0730 - val_accuracy: 0.2054 - val_loss: 3.1779
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 33s 643ms/step - accuracy: 0.7565 - loss: 0.6947 - val_accuracy: 0.2738 - val_loss: 4.8775
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 35s 677ms/step - accuracy: 0.8407 - loss: 0.5045 - val_accuracy: 0.2078 - val_loss: 7.8093
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 34s 658ms/step - accuracy: 0.9144 - loss: 0.2706 - val_accuracy: 0.2372 - val_loss: 6.8697
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 47s 900ms/step - accuracy: 0.9425 - loss: 0.1738 - val_accuracy: 0.2567 - val_loss: 5.9043
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 35s 670ms/step - accuracy: 0.9792 - loss: 0.0830 - val_accuracy: 0.2445 - val_loss: 8.1382
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 38s 738ms/step - accuracy: 0.9850 - loss: 0.0595 - val_accuracy: 0.251

### Paso 6: Pobrar el modelo

In [ ]:
# Probar el modelo
import numpy as np
from tensorflow.keras.preprocessing import image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión de batch
    img_array /= 255.0  # Normalizar
    return img_array

def predict_image(model, img_path):
    img_array = load_and_preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

# Probar con una imagen de prueba

import os

img_path = r"C:\IA\CourseIA\Unidad4\Tarea1\pruebados.jpg"
if os.path.exists(img_path):
    img_array = load_and_preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    confidence = np.max(predictions) * 100
    # Obtener el nombre de la clase
    class_labels = list(train_generator.class_indices.keys())
    class_name = class_labels[predicted_class[0]]
    print(f"Predicted class: {predicted_class[0]} ({class_name})")
    print(f"Porcentaje de acierto: {confidence:.2f}%")
else:
    print(f"Image not found: {img_path}")   
# Guardar el modelo



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted class: 3 (happy)
Porcentaje de acierto: 97.76%
